In [1]:
cd RTM/src/

/home/kaixin1/Project/RTM/src


In [2]:
import torch
torch.__version__

'0.4.1.post2'

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os
import json
import cv2
import numpy as np
import time
from progress.bar import Bar
import torch

from opts import opts
from logger import Logger
from utils.utils import AverageMeter
from datasets.dataset_factory import dataset_factory

In [5]:
%cd lib/utils
from detectors.detector_factory import detector_factory

/home/kaixin1/Project/RTM/src/lib/utils


In [6]:
import detectors.detector_factory 
import imp
imp.reload(detectors.detector_factory)
from detectors.detector_factory import detector_factory

In [7]:
args = ['ddd', '--exp_id', '3dop', '--dataset', 'kitti',\
'--kitti_split', '3dop' ,'--load_model','../models/model_last.pth' ,'--arch', 'resFP_18']

In [8]:
opt = opts().parse(args)

opt.gpus [0]
Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /home/kaixin1/Project/RTM/src/lib/../../exp/ddd/3dop


In [9]:
cd ../..

/home/kaixin1/Project/RTM/src


In [10]:
Dataset = dataset_factory[opt.dataset]
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)
Logger(opt)
Detector = detector_factory[opt.task]

split = 'val' if not opt.trainval else 'test'
dataset = Dataset(opt, split)
detector = Detector(opt)

heads {'hm': 3, 'vertex_hm': 9, 'vertex_coordinate': 18, 'dep': 1, 'rot': 8, 'dim': 3, 'wh': 2, 'reg': 2, 'vertex_reg_offset': 2}
Namespace(K=100, aggr_weight=0.0, agnostic_ex=False, arch='resFP_18', aug_ddd=0.5, aug_rot=0, batch_size=32, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[32], data_dir='/home/kaixin1/Project/RTM/src/lib/../../data', dataset='kitti', debug=0, debug_dir='/home/kaixin1/Project/RTM/src/lib/../../exp/ddd/3dop/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/home/kaixin1/Project/RTM/src/lib/../../exp/ddd', exp_id='3dop', fix_res=True, flip=0.5, flip_test=False, gpus=[0], gpus_str='0', head_conv=64, heads={'hm': 3, 'vertex_hm': 9, 'vertex_coordinate': 18, 'dep': 1, 'rot': 8, 'dim': 3, 'wh': 2, 'reg': 2, 'vertex_reg_off

In [11]:
  results = {}
  num_iters = len(dataset)
  bar = Bar('{}'.format(opt.exp_id), max=num_iters)
  time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge']
  avg_time_stats = {t: AverageMeter() for t in time_stats}
  for ind in range(8):
    img_id = dataset.images[ind]
    img_info = dataset.coco.loadImgs(ids=[img_id])[0]
    img_path = os.path.join(dataset.img_dir, img_info['file_name'])
    print(img_path)
    if opt.task == 'ddd':
      ret = detector.run(img_path, img_info['calib'])
    else:
      ret = detector.run(img_path)
    
    results[img_id] = ret['results']

    Bar.suffix = '[{0}/{1}]|Tot: {total:} |ETA: {eta:} '.format(
                   ind, num_iters, total=bar.elapsed_td, eta=bar.eta_td)
    for t in avg_time_stats:
      avg_time_stats[t].update(ret[t])
      Bar.suffix = Bar.suffix + '|{} {:.3f} '.format(t, avg_time_stats[t].avg)
    bar.next()
  bar.finish()
  dataset.run_eval(results, opt.save_dir)

/home/kaixin1/Project/RTM/src/lib/../../data/kitti/images/trainval/000001.png


/home/kaixin1/anaconda3/envs/CenterNet/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


input1 [  1.6407814   1.4907396   3.953827    1.6425533 -15.6911      2.2080824
  55.3991   ]
input2 [406.8916  390.3306  402.83105 415.41138 406.97302 399.07245 409.64877
 415.61197 405.62866 203.5897  201.34924 200.7236  203.0773  184.20221
 184.17953 183.96516 184.01463 192.92787]
input3 [721.5377 609.5593 172.854 ]
score [0.6099873]
hm_score [0.8818773  0.59903467 0.80183125 0.88541454 0.6180065  0.33696684
 0.38163295 0.706965   0.5214315 ]
optimized_result [  1.6407814    1.17173602   3.9538269    1.68382222 -17.97143979
   2.35540932  63.14167664]
--------------------------------------------------
/home/kaixin1/Project/RTM/src/lib/../../data/kitti/images/trainval/000002.png
/home/kaixin1/Project/RTM/src/lib/../../data/kitti/images/trainval/000004.png
input1 [  1.6506581   1.4846267   3.9849634   1.4771068 -15.95024     2.2399638
  38.53309  ]
input2 [312.85092 304.53857 316.4663  322.7816  312.489   304.54242 318.65335
 312.41864 312.38422 216.3148  216.08372 212.88367 216.33124

In [11]:
opt.save_dir

'/home/kaixin1/Project/RTM/src/lib/../../exp/ddd/3dop'

In [13]:
from models.model import create_model, load_model
from utils.image import get_affine_transform
from utils.debugger import Debugger

In [14]:
self = detector
image_or_path_or_tensor, meta = img_path, img_info['calib']

In [15]:
len(dataset.images)

3769

In [16]:
    load_time, pre_time, net_time, dec_time, post_time = 0, 0, 0, 0, 0
    merge_time, tot_time = 0, 0
    debugger = Debugger(dataset=self.opt.dataset, ipynb=(self.opt.debug==3),
                        theme=self.opt.debugger_theme)
    start_time = time.time()
    pre_processed = False
    if isinstance(image_or_path_or_tensor, np.ndarray):
      image = image_or_path_or_tensor
    elif type(image_or_path_or_tensor) == type (''): 
      image = cv2.imread(image_or_path_or_tensor)
    else:
      image = image_or_path_or_tensor['image'][0].numpy()
      pre_processed_images = image_or_path_or_tensor
      pre_processed = True
    
    loaded_time = time.time()
    load_time += (loaded_time - start_time)
    
    detections = []
    for scale in self.scales:
      scale_start_time = time.time()
      if not pre_processed:
        images, meta = self.pre_process(image, scale, meta)
      else:
        # import pdb; pdb.set_trace()
        images = pre_processed_images['images'][scale][0]
        meta = pre_processed_images['meta'][scale]
        meta = {k: v.numpy()[0] for k, v in meta.items()}
      images = images.to(self.opt.device)
      torch.cuda.synchronize()
      pre_process_time = time.time()
      pre_time += pre_process_time - scale_start_time
      
      output, dets1, forward_time = self.process(images, return_time=True)
      torch.cuda.synchronize()
      net_time += forward_time - pre_process_time
      decode_time = time.time()
      dec_time += decode_time - forward_time
      
      if self.opt.debug >= 2:
        self.debug(debugger, images, dets, output, scale)
      print("before post_process",dets1)
      dets = self.post_process(dets1, meta, scale)
      print("after post_process",dets)
      torch.cuda.synchronize()
      post_process_time = time.time()
      post_time += post_process_time - decode_time

      detections.append(dets)
      print("detections",detections)
    
    results = self.merge_outputs(detections)
    torch.cuda.synchronize()
    end_time = time.time()
    merge_time += end_time - post_process_time
    tot_time += end_time - start_time

    if self.opt.debug >= 1:
      self.show_results(debugger, image, results)

before post_process tensor([[[104.6009,  49.1335,   0.6100,  ..., 104.5098,  49.3985,   1.0000],
         [173.6738,  59.4021,   0.0466,  ..., 174.3945,  59.6291,   1.0000],
         [170.5235,  61.5215,   0.0426,  ..., 171.2695,  61.8699,   1.0000],
         ...,
         [318.5414,  61.2688,   0.0123,  ..., 319.4401,  61.0261,   0.0000],
         [ 97.7459,  17.5811,   0.0122,  ...,  98.4917,  18.1051,   1.0000],
         [317.6130,  69.4407,   0.0122,  ..., 318.5959,  69.0912,   2.0000]]],
       device='cuda:0')
after post_process {1: array([[ 1.91754818e+00,  3.89428375e+02,  1.80761932e+02,
         4.22536469e+02,  2.03036652e+02,  1.49073958e+00,
         1.64078140e+00,  3.95382690e+00, -1.56911001e+01,
         2.20808244e+00,  5.53991013e+01,  1.64255333e+00,
         2.75796149e-02],
       [ 1.65812445e+00,  1.19267310e+03,  2.47844589e+02,
         1.27277100e+03,  3.24823639e+02,  1.54470384e+00,
         1.27635038e+00,  2.93841314e+00,  3.25436926e+00,
         1.37624

In [17]:
dets1[0][0]

tensor([104.6009,  49.1335,   0.6100,   0.0939,  -0.0759,   0.1654,   0.9550,
          0.0229,   0.0204,   0.3619,   1.0014,  55.4018,   1.4907,   1.6408,
          3.9538,   8.5303,   5.4299, 104.8352,  52.1455, 100.5683,  51.5682,
        103.7890,  51.4070, 107.0303,  52.0135, 104.8562,  47.1503, 102.8206,
         47.1445, 105.5456,  47.0893, 107.0820,  47.1020, 104.5098,  49.3985,
          1.0000], device='cuda:0')

In [19]:
dets[2]

array([[  1.9175482, 389.42838  , 180.76193  , 422.53647  , 203.03665  ,
          1.4907396,   1.6407814,   3.953827 , -15.6911   ,   2.2080824,
         55.3991   ,   1.6425533,   0.6099873]], dtype=float32)

In [42]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [29]:
def get_pred_depth(depth):
  return depth

def get_alpha(rot):
  # output: (B, 8) [bin1_cls[0], bin1_cls[1], bin1_sin, bin1_cos, 
  #                 bin2_cls[0], bin2_cls[1], bin2_sin, bin2_cos]
  # return rot[:, 0]
  idx = rot[:, 1] > rot[:, 5]
  alpha1 = np.arctan(rot[:, 2] / rot[:, 3]) + (-0.5 * np.pi)
  alpha2 = np.arctan(rot[:, 6] / rot[:, 7]) + ( 0.5 * np.pi)
  return alpha1 * idx + alpha2 * (1 - idx)

In [26]:


def get_3rd_point(a, b):
    direct = a - b
    return b + np.array([-direct[1], direct[0]], dtype=np.float32)


def get_dir(src_point, rot_rad):
    sn, cs = np.sin(rot_rad), np.cos(rot_rad)

    src_result = [0, 0]
    src_result[0] = src_point[0] * cs - src_point[1] * sn
    src_result[1] = src_point[0] * sn + src_point[1] * cs

    return src_result


def crop(img, center, scale, output_size, rot=0):
    trans = get_affine_transform(center, scale, rot, output_size)

    dst_img = cv2.warpAffine(img,
                             trans,
                             (int(output_size[0]), int(output_size[1])),
                             flags=cv2.INTER_LINEAR)

    return dst_img


In [48]:
with torch.no_grad():
    torch.cuda.synchronize()
    output = self.model(images)[-1]
    output['hm'] = output['hm'].sigmoid_()
    output['dep'] = 1. / (output['dep'].sigmoid() + 1e-6) - 1.
    wh = output['wh'] if self.opt.reg_bbox else None
    reg = output['reg'] if self.opt.reg_offset else None
    output['vertex_hm'] = output['vertex_hm'].sigmoid_()
    torch.cuda.synchronize()
    forward_time = time.time()

    heat, rot, depth, dim,vertex_coordinate,vertex_hm, vertex_reg_offset = \
    output['hm'], output['rot'], output['dep'],output['dim'], output['vertex_coordinate'], \
    output['vertex_hm'], output['vertex_reg_offset']
    K=self.opt.K
    
    batch, cat, height, width = heat.size()
    # heat = torch.sigmoid(heat)
    # perform nms on heatmaps
    heat = _nms(heat)

    scores, inds, clses, ys, xs = _topk(heat, K=K)
    if reg is not None:
      reg = _transpose_and_gather_feat(reg, inds)
      reg = reg.view(batch, K, 2)
      xs = xs.view(batch, K, 1) + reg[:, :, 0:1]
      ys = ys.view(batch, K, 1) + reg[:, :, 1:2]
    else:
      xs = xs.view(batch, K, 1) + 0.5
      ys = ys.view(batch, K, 1) + 0.5

    vertex_coordinate = _transpose_and_gather_feat(vertex_coordinate, inds)
    vertex_coordinate = vertex_coordinate.view(batch, K, 9 * 2)
    vertex_coordinate[..., ::2] += xs.view(batch, K, 1).expand(batch, K, 9)
    vertex_coordinate[..., 1::2] += ys.view(batch, K, 1).expand(batch, K, 9)
    rot = _transpose_and_gather_feat(rot, inds)
    rot = rot.view(batch, K, 8)
    depth = _transpose_and_gather_feat(depth, inds)
    depth = depth.view(batch, K, 1)
    dim = _transpose_and_gather_feat(dim, inds)
    dim = dim.view(batch, K, 3)
    clses  = clses.view(batch, K, 1).float()
    scores = scores.view(batch, K, 1)
    xs = xs.view(batch, K, 1)
    ys = ys.view(batch, K, 1)
    if wh is not None:
      wh = _transpose_and_gather_feat(wh, inds)
      wh = wh.view(batch, K, 2)
    bboxes = torch.cat([xs - wh[..., 0:1] / 2, 
                      ys - wh[..., 1:2] / 2,
                      xs + wh[..., 0:1] / 2, 
                      ys + wh[..., 1:2] / 2], dim=2)
    vertex_hm = _nms(vertex_hm)
    thresh = 0.1
    vertex_coordinate = vertex_coordinate.view(batch, K, 9, 2).permute(0, 2, 1, 3).contiguous() # b x J x K x 2
    reg_vertex_coordinate = vertex_coordinate.unsqueeze(3).expand(batch, 9, K, K, 2)
    hm_score, hm_inds, hm_ys, hm_xs = _topk_channel(vertex_hm, K=K) 
    if vertex_reg_offset is not None:
        vertex_reg_offset = _transpose_and_gather_feat(
            vertex_reg_offset, hm_inds.view(batch, -1))
        vertex_reg_offset = vertex_reg_offset.view(batch, 9, K, 2)
        hm_xs = hm_xs + vertex_reg_offset[:, :, :, 0]
        hm_ys = hm_ys + vertex_reg_offset[:, :, :, 1]
    else:
        hm_xs = hm_xs + 0.5
        hm_ys = hm_ys + 0.5 
    mask = (hm_score > thresh).float()
    hm_score = (1 - mask) * -1 + mask * hm_score
    hm_ys = (1 - mask) * (-10000) + mask * hm_ys
    hm_xs = (1 - mask) * (-10000) + mask * hm_xs
    hm_kps = torch.stack([hm_xs, hm_ys], dim=-1).unsqueeze(
          2).expand(batch, 9, K, K, 2)
    dist = (((reg_vertex_coordinate - hm_kps) ** 2).sum(dim=4) ** 0.5)
    min_dist, min_ind = dist.min(dim=3) # b x 9 x K
    hm_score = hm_score.gather(2, min_ind).unsqueeze(-1) # b x J x K x 1
    min_dist = min_dist.unsqueeze(-1)
    min_ind = min_ind.view(batch, 9, K, 1, 1).expand(
          batch, 9, K, 1, 2)
    hm_kps = hm_kps.gather(3, min_ind)
    hm_kps = hm_kps.view(batch, 9, K, 2)
    l = bboxes[:, :, 0].view(batch, 1, K, 1).expand(batch, 9, K, 1)
    t = bboxes[:, :, 1].view(batch, 1, K, 1).expand(batch, 9, K, 1)     
    r = bboxes[:, :, 2].view(batch, 1, K, 1).expand(batch, 9, K, 1)
    b = bboxes[:, :, 3].view(batch, 1, K, 1).expand(batch, 9, K, 1)
    mask = (hm_kps[..., 0:1] < l) + (hm_kps[..., 0:1] > r) + \
             (hm_kps[..., 1:2] < t) + (hm_kps[..., 1:2] > b) + \
             (hm_score < thresh) + (min_dist > (torch.max(b - t, r - l) * 0.3))
    mask = (mask > 0).float().expand(batch, 9, K, 2)
    vertex_coordinate = (1 - mask) * hm_kps + mask * vertex_coordinate
    vertex_coordinate = vertex_coordinate.permute(0, 2, 1, 3).contiguous().view(
          batch, K, 9 * 2)

    detections = torch.cat([xs, ys, scores, rot, depth, dim, wh, vertex_coordinate, clses], dim=2)

In [50]:
detections[0][0]

tensor([254.4854,  91.4898,   0.1312,  -0.1453,   0.1416,  -0.0374,   0.6834,
         -0.0826,   0.1531,   0.2100,   0.6800,  23.6818,   1.4992,   1.5596,
          3.6431,  11.7349,  10.8012, 254.3587,  96.7891, 252.9255,  96.7456,
        256.5332,  96.7633, 256.9821,  96.8143, 254.2609,  86.2760, 252.9080,
         86.3763, 256.5728,  86.2066, 256.9952,  86.0962, 254.4803,  91.4762,
          1.0000], device='cuda:0')

In [46]:
import torch
import torch.nn as nn
# from .utils import _gather_feat, _transpose_and_gather_feat

def _nms(heat, kernel=3):
    pad = (kernel - 1) // 2

    hmax = nn.functional.max_pool2d(
        heat, (kernel, kernel), stride=1, padding=pad)
    keep = (hmax == heat).float()
    return heat * keep

def _left_aggregate(heat):
    '''
        heat: batchsize x channels x h x w
    '''
    shape = heat.shape 
    heat = heat.reshape(-1, heat.shape[3])
    heat = heat.transpose(1, 0).contiguous()
    ret = heat.clone()
    for i in range(1, heat.shape[0]):
        inds = (heat[i] >= heat[i - 1])
        ret[i] += ret[i - 1] * inds.float()
    return (ret - heat).transpose(1, 0).reshape(shape) 

def _right_aggregate(heat):
    '''
        heat: batchsize x channels x h x w
    '''
    shape = heat.shape 
    heat = heat.reshape(-1, heat.shape[3])
    heat = heat.transpose(1, 0).contiguous()
    ret = heat.clone()
    for i in range(heat.shape[0] - 2, -1, -1):
        inds = (heat[i] >= heat[i +1])
        ret[i] += ret[i + 1] * inds.float()
    return (ret - heat).transpose(1, 0).reshape(shape) 

def _top_aggregate(heat):
    '''
        heat: batchsize x channels x h x w
    '''
    heat = heat.transpose(3, 2) 
    shape = heat.shape
    heat = heat.reshape(-1, heat.shape[3])
    heat = heat.transpose(1, 0).contiguous()
    ret = heat.clone()
    for i in range(1, heat.shape[0]):
        inds = (heat[i] >= heat[i - 1])
        ret[i] += ret[i - 1] * inds.float()
    return (ret - heat).transpose(1, 0).reshape(shape).transpose(3, 2)

def _bottom_aggregate(heat):
    '''
        heat: batchsize x channels x h x w
    '''
    heat = heat.transpose(3, 2) 
    shape = heat.shape
    heat = heat.reshape(-1, heat.shape[3])
    heat = heat.transpose(1, 0).contiguous()
    ret = heat.clone()
    for i in range(heat.shape[0] - 2, -1, -1):
        inds = (heat[i] >= heat[i + 1])
        ret[i] += ret[i + 1] * inds.float()
    return (ret - heat).transpose(1, 0).reshape(shape).transpose(3, 2)

def _h_aggregate(heat, aggr_weight=0.1):
    return aggr_weight * _left_aggregate(heat) + \
           aggr_weight * _right_aggregate(heat) + heat

def _v_aggregate(heat, aggr_weight=0.1):
    return aggr_weight * _top_aggregate(heat) + \
           aggr_weight * _bottom_aggregate(heat) + heat

'''
# Slow for large number of categories
def _topk(scores, K=40):
    batch, cat, height, width = scores.size()
    topk_scores, topk_inds = torch.topk(scores.view(batch, -1), K)

    topk_clses = (topk_inds / (height * width)).int()

    topk_inds = topk_inds % (height * width)
    topk_ys   = (topk_inds / width).int().float()
    topk_xs   = (topk_inds % width).int().float()
    return topk_scores, topk_inds, topk_clses, topk_ys, topk_xs
'''
def _topk_channel(scores, K=40):
      batch, cat, height, width = scores.size()
      
      topk_scores, topk_inds = torch.topk(scores.view(batch, cat, -1), K)

      topk_inds = topk_inds % (height * width)
      topk_ys   = (topk_inds / width).int().float()
      topk_xs   = (topk_inds % width).int().float()

      return topk_scores, topk_inds, topk_ys, topk_xs

def _topk(scores, K=40):
    batch, cat, height, width = scores.size()
      
    topk_scores, topk_inds = torch.topk(scores.view(batch, cat, -1), K)

    topk_inds = topk_inds % (height * width)
    topk_ys   = (topk_inds / width).int().float()
    topk_xs   = (topk_inds % width).int().float()
      
    topk_score, topk_ind = torch.topk(topk_scores.view(batch, -1), K)
    topk_clses = (topk_ind / K).int()
    topk_inds = _gather_feat(
        topk_inds.view(batch, -1, 1), topk_ind).view(batch, K)
    topk_ys = _gather_feat(topk_ys.view(batch, -1, 1), topk_ind).view(batch, K)
    topk_xs = _gather_feat(topk_xs.view(batch, -1, 1), topk_ind).view(batch, K)

    return topk_score, topk_inds, topk_clses, topk_ys, topk_xs

def _sigmoid(x):
  y = torch.clamp(x.sigmoid_(), min=1e-4, max=1-1e-4)
  return y

def _gather_feat(feat, ind, mask=None):
    dim  = feat.size(2)
    ind  = ind.unsqueeze(2).expand(ind.size(0), ind.size(1), dim)
    feat = feat.gather(1, ind)
    if mask is not None:
        mask = mask.unsqueeze(2).expand_as(feat)
        feat = feat[mask]
        feat = feat.view(-1, dim)
    return feat

def _transpose_and_gather_feat(feat, ind):
    feat = feat.permute(0, 2, 3, 1).contiguous()
    feat = feat.view(feat.size(0), -1, feat.size(3))
    feat = _gather_feat(feat, ind)
    return feat

def flip_tensor(x):
    return torch.flip(x, [3])
    # tmp = x.detach().cpu().numpy()[..., ::-1].copy()
    # return torch.from_numpy(tmp).to(x.device)


In [43]:
cd models

/home/kaixin1/Project/RTM/src/lib/models


In [23]:

def transform_preds(coords, center, scale, output_size):
    target_coords = np.zeros(coords.shape)
    trans = get_affine_transform(center, scale, 0, output_size, inv=1)
    for p in range(coords.shape[0]):
        target_coords[p, 0:2] = affine_transform(coords[p, 0:2], trans)
    return target_coords


def get_affine_transform(center,
                         scale,
                         rot,
                         output_size,
                         shift=np.array([0, 0], dtype=np.float32),
                         inv=0):
    if not isinstance(scale, np.ndarray) and not isinstance(scale, list):
        scale = np.array([scale, scale], dtype=np.float32)

    scale_tmp = scale
    src_w = scale_tmp[0]
    dst_w = output_size[0]
    dst_h = output_size[1]

    rot_rad = np.pi * rot / 180
    src_dir = get_dir([0, src_w * -0.5], rot_rad)
    dst_dir = np.array([0, dst_w * -0.5], np.float32)

    src = np.zeros((3, 2), dtype=np.float32)
    dst = np.zeros((3, 2), dtype=np.float32)
    src[0, :] = center + scale_tmp * shift
    src[1, :] = center + src_dir + scale_tmp * shift
    dst[0, :] = [dst_w * 0.5, dst_h * 0.5]
    dst[1, :] = np.array([dst_w * 0.5, dst_h * 0.5], np.float32) + dst_dir

    src[2:, :] = get_3rd_point(src[0, :], src[1, :])
    dst[2:, :] = get_3rd_point(dst[0, :], dst[1, :])

    if inv:
        trans = cv2.getAffineTransform(np.float32(dst), np.float32(src))
    else:
        trans = cv2.getAffineTransform(np.float32(src), np.float32(dst))

    return trans


def affine_transform(pt, t):
    new_pt = np.array([pt[0], pt[1], 1.], dtype=np.float32).T
    new_pt = np.dot(t, new_pt)
    return new_pt[:2]


In [22]:
a,b,c=np.array([1,2,3])

In [23]:
b

2